In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd
df = pd.read_excel("data.xls")
li = df[df.isnull().T.any()].index
for i in li:
    df.drop(index=i,axis = 1,inplace = True)
df[df.isnull().T.any()]

,tweet,polarity,subjectivity,vader_neg,vader_neu,vader_pos,vader_compound


In [4]:
df['vd_new']=5
df.loc[df['vader_compound']==0,['vd_new']]=0
df.loc[df['vader_compound']>0,['vd_new']]=1
df.loc[df['vader_compound']<0,['vd_new']]=-1
df['vd_new'].value_counts()


 1    3294
 0    2936
-1    1427
Name: vd_new, dtype: int64

In [5]:
X = df['tweet']
Y = df['vd_new']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)
X_train

1766    Video Dubai residents queue up to receive COVI...
7267    This is MY better half and I m supposed to get...
1430    Neither will we PfizerBioNTech has produced an...
3551    uk Apart from the oxfordvaccine the Moderna Pf...
2405    Are you counted as been vaccinated after the f...
                              ...                        
4063    The BMJ 23 patients have died in Norway shortl...
1346    Joe Biden receives his first dose of Pfizer CO...
3457    Laboratory tests have shown the PfizerBioNTech...
7537    My hubby is fully vaccinated Thank you Science...
3585    Anyone in Ireland or anywhere else thinking of...
Name: tweet, Length: 6125, dtype: object

In [7]:
num_features = 200
vect = CountVectorizer(max_features=num_features)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1532x200 sparse matrix of type '<class 'numpy.int64'>'
	with 13212 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# instantiate a logistic regression model
logreg = LogisticRegression(multi_class='multinomial', solver="lbfgs", C=10, random_state=123)

In [9]:
%time logreg.fit(X_train_dtm,y_train)

CPU times: user 239 ms, sys: 8.87 ms, total: 248 ms
Wall time: 259 ms


/Users/linqiaoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, multi_class='multinomial', random_state=123)

In [10]:
from sklearn.metrics import classification_report
y_test_pred = logreg.predict(X_test_dtm)
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))
print(classification_report(y_test,y_test_pred))

Accuracy:0.6298955613577023
Precision:0.6298955613577023
Recall:0.6298955613577023
              precision    recall  f1-score   support

          -1       0.56      0.34      0.43       294
           0       0.59      0.77      0.67       598
           1       0.70      0.63      0.66       640

    accuracy                           0.63      1532
   macro avg       0.62      0.58      0.59      1532
weighted avg       0.63      0.63      0.62      1532



In [11]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def custom_preprocessor(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    words = word_tokenize(text)
    words = [i for i in words if i not in stopwords.words()]
    text = ' '.join(words)
    
    return text

In [12]:
df['tweet_clean']=df['tweet'].apply(custom_preprocessor)
X = df['tweet_clean']
Y = df['vd_new']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)
X_train

1766    video dubai residents queue receive vaccinatio...
7267    better half supposed get first pfizerbiontech ...
1430    neither pfizerbiontech produced experimental m...
3551    uk apart oxfordvaccine moderna pfizerbiontech ...
2405                     counted vaccinated first jab sec
                              ...                        
4063    bmj patients died norway shortly receiving pfi...
1346    joe biden receives first dose pfizer covid vac...
3457    laboratory tests shown pfizerbiontech vaccine ...
7537    hubby fully vaccinated thank science pfizerbio...
3585    anyone ireland anywhere else thinking receivin...
Name: tweet_clean, Length: 6125, dtype: object

In [13]:
vect = CountVectorizer(ngram_range=(1,2))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_train_dtm

<6125x39898 sparse matrix of type '<class 'numpy.int64'>'
	with 100109 stored elements in Compressed Sparse Row format>

In [14]:
%%time

logreg = LogisticRegression()
logreg.fit(X_train_dtm,y_train)


CPU times: user 2.88 s, sys: 52.1 ms, total: 2.93 s
Wall time: 1.58 s


/Users/linqiaoyi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
y_test_pred = logreg.predict(X_test_dtm)
print("Accuracy:{}".format(accuracy_score(y_test, y_test_pred)))
print("Precision:{}".format(precision_score(y_test, y_test_pred,average='micro')))
print("Recall:{}".format(recall_score(y_test, y_test_pred,average='micro')))
print(classification_report(y_test,y_test_pred))

Accuracy:0.7663185378590078
Precision:0.7663185378590078
Recall:0.7663185378590078
              precision    recall  f1-score   support

          -1       0.78      0.45      0.57       294
           0       0.71      0.92      0.80       598
           1       0.84      0.77      0.80       640

    accuracy                           0.77      1532
   macro avg       0.78      0.71      0.72      1532
weighted avg       0.78      0.77      0.76      1532



In [18]:
import numpy as np
df1 = pd.read_csv('Finished_vaccination_tweets_手动标记.csv',encoding='ISO-8859-1')
df1 = df1[(df1['code-attitude'] == '0') | (df1['code-attitude'] == '1') | (df1['code-attitude'] == '-1')]
df1['text']

2       Facts are immutable, Senator, even when you're...
3       Explain to me again why we need a vaccine @Bor...
4       Does anyone have any useful advice/guidance fo...
5       it is a bit sad to claim the fame for success ...
6       There have not been many bright days in 2020 b...
                              ...                        
3101    #PublicHealth #COVID19 #Modernavaccine #Modern...
3102    @crashoverrideee #COVID19 Vaccine Update for #...
3103    Dr Fun's I Feel Good - My effort at a PSA, ple...
3104    @Writer_DG I got dose #2 Friday. Have to admit...
3105    Feeling very privileged to have had my first c...
Name: text, Length: 247, dtype: object

In [19]:
df1['text'] = pd.DataFrame(df1['text'].astype(str))
df1['text_clean']=df1['text'].apply(custom_preprocessor)
t = vect.transform(df1['text_clean'])
y_test_pred = logreg.predict(t)
df1['code-attitude'] = pd.DataFrame(df1['code-attitude'].astype(int))
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy:{}".format(accuracy_score(df1['code-attitude'], y_test_pred)))


Accuracy:0.6396761133603239
